In [ ]:
#!sudo pkill pigpiod
#!sudo pigpiod -p 8889 ### Start PiGPIO Server / Deamon          http://abyz.me.uk/rpi/pigpio/pigpiod.html

In [ ]:
import io, os, sys, ssl, time, json, socket, struct, random, datetime, threading
import serial, pigpio, picamera
from adafruit_motorkit import MotorKit

server_ip = 'guoxiaokang.net'
img_port = 54321
ctrl_port = 54322 
sensor_port = 54324

server_cert_self_signed  = 'secret/server.crt' 
client_certs_self_signed = 'secret/rover.crt'
client_key_self_signed   = 'secret/rover.key'

# Establish connection with server
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert_self_signed)
context.load_cert_chain(certfile=client_certs_self_signed, keyfile=client_key_self_signed)

In [ ]:
kit = MotorKit() 
pi = pigpio.pi(port=8889) ### Start PiGPIO Client
rotation_pin, elevation_pin = 17, 27  # {sky: 1300 ~ 1700 ~ 2300 & front: 500 ~ 2500} 
rotate = lambda val: pi.set_servo_pulsewidth(rotation_pin, val) 
elevate = lambda val: pi.set_servo_pulsewidth(elevation_pin, val) 

In [ ]:
rotate(2200) # 600 ~ back 1000 ~ front 2200 ~ 2500

In [ ]:
elevate(1850)

In [ ]:
for i in range(700, 2300, 100):
    elevate(i) # down [1500  ~ 1800 ~ 2350
    time.sleep(1)

In [ ]:
time.sleep(3)
kit.motor3.throttle = 1.0
kit.motor4.throttle = 1.0
time.sleep(1)
kit.motor3.throttle = 0
kit.motor4.throttle = 0

In [ ]:
!pip3 install evdev

In [ ]:
from evdev import InputDevice, ecodes
device = InputDevice('/dev/input/event0')   # Events for Xbox One S Wireless Controller

In [ ]:
class XboxController():
    def __init__(self):
        self.rotation_rear     = 1000 # back 1000 ~ front 2200
        self.rotation_front    = 2200 
        self.elevation_horiz   = 1850 
        self.gimbal_sensitivity= 0.01 
        self.rot = self.rotation_front
        self.ele = self.elevation_horiz 
    def action(self, code, value):    # 31000 32000  1111  
        if code == 0 and (value > 37000 or value < 34000):     # 0    ->  ABS_X      ->  trn 
            increment = (value - 33000)*self.turn_sensitivity   
        elif code == 1 and (value > 32000 or value < 31000):   # 1    ->  ABS_Y      ->  go 
            increment = (value - 31500)*self.turn_sensitivity  
        elif code == 2 and (value > 34000 or value < 31000):   # 2    ->  ABS_Z      ->  rot  
            self.rot += (value - 33000)*self.gimbal_sensitivity 
            if   self.rot > self.rotation_front: self.rot = self.rotation_front
            elif self.rot < self.rotation_rear : self.rot = self.rotation_rear
            return code, int(self.rot) 
        elif code == 5 and (value > 35000 or value < 33000):   # 5    ->  ABS_RZ     ->  ele     
            self.ele -= (value - 33000)*self.gimbal_sensitivity 
            if   self.ele > 2500: self.ele = 2500
            elif self.ele < 500 : self.ele = 500
            return code, int(self.ele) 
        return 999, None 
        
xbox = XboxController() 

In [ ]:
for event in device.read_loop(): 
    if event.type in [ecodes.EV_ABS, ecodes.EV_KEY]: 
        code, value = xbox.action(event.code, event.value)
        if code == 2:
            rotate(value)
        elif code == 5:
            elevate(value)

In [ ]:
class XboxController():
    def __init__(self):
        self.rotation_rear     = 1000 # back 1000 ~ front 2200
        self.rotation_front    = 2200 
        self.elevation_horiz   = 1850 
        self.gimbal_sensitivity= 0.01
        self.rot = self.rotation_front
        self.ele = self.elevation_horiz 
    def action(self, code, value):   
        if code == 0 and (value > 37000 or value < 34000):     # 0    ->  ABS_X      ->  trn 
            increment = (value - 33000)*self.turn_sensitivity
            self.top += increment
            if   self.top > self.turn_right_top: self.top = self.turn_right_top
            elif self.top < self.turn_left_top : self.top = self.turn_left_top 
            self.bot -= increment
            if   self.bot > self.turn_left_bottom : self.bot = self.turn_left_bottom
            elif self.bot < self.turn_right_bottom: self.bot = self.turn_right_bottom   
            sc_rover.send(struct.pack('<Hi', code, int(self.top)))  
        elif code == 2 and (value > 34000 or value < 31000):   # 2    ->  ABS_Z      ->  rot  
            self.rot -= (value - 33000)*self.gimbal_sensitivity
            if   self.rot > self.rotation_rear : self.rot = self.rotation_rear 
            elif self.rot < self.rotation_front: self.rot = self.rotation_front
            sc_rover.send(struct.pack('<Hi', code, int(self.rot)))
        elif code == 5 and (value > 35000 or value < 33000):   # 5    ->  ABS_RZ     ->  ele     
            self.ele += (value - 33000)*self.gimbal_sensitivity 
            if   self.ele > self.elevation_horiz: self.ele = self.elevation_horiz
            elif self.ele < self.elevation_sky  : self.ele = self.elevation_sky 
            sc_rover.send(struct.pack('<Hi', code, int(self.ele)))
        elif code ==  10:
            #if value 
            sc_rover.send(struct.pack('<Hi', code, int(value/1023*255)))     # 10   ->  ABS_BRAKE  ->  rev  
        elif code ==   9: 
            sc_rover.send(struct.pack('<Hi', code, int(value/1023*255)))     # 9    ->  ABS_GAS    ->  drv   
        elif code ==  16: # sensitivity
            enum = [0.00001, 0.0001, 0.001, 0.01]
            idx  = enum.index(self.turn_sensitivity) 
            idx += value 
            if idx >= 0 and idx < len(enum): 
                self.turn_sensitivity = enum[idx]  
        elif code ==  17: # sensitivity
            enum = [0.00001, 0.0001, 0.001, 0.01]
            idx  = enum.index(self.gimbal_sensitivity) 
            idx -= value 
            if idx >= 0 and idx < len(enum): 
                self.gimbal_sensitivity = enum[idx] 
        elif code == 304: sc_rover.send(struct.pack('<Hi', code, 1350))     # 304  ->  BTN_A      ->  rst
        elif code == 305: 
            global Sensor
            if Sensor['focus'] == 'true': 
                Sensor['focus'] = 'false'
            else:
                Sensor['focus'] = 'true'
xbox = XboxController() 

In [ ]:
code2name = {}
for type_, code_ in device.capabilities(verbose=True, absinfo=False).items(): 
    if type_[0] in ['EV_KEY', 'EV_ABS']:
        for name, code in code_:  
            if isinstance(name,list): 
                if   'BTN_A' in name: code2name[code] = 'BTN_A'  
                elif 'BTN_B' in name: code2name[code] = 'BTN_B'   
                elif 'BTN_X' in name: code2name[code] = 'BTN_X'  
                elif 'BTN_Y' in name: code2name[code] = 'BTN_Y'  
                continue
            code2name[code] = name
print(code2name)

In [ ]:
if 1:
    #def ctrd():
    l = struct.calcsize('<hi')
    while True:
        try:
            print('Try !')
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            ctrl_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            ctrl_ss.settimeout(30)
            print('Try to connect the server')
            ctrl_ss.connect((server_ip, ctrl_port))
            print('Succeed a connection')
            while True: 
                code, value = struct.unpack('<hi', ctrl_ss.recv(l)) 
                continue 
                elif code ==  2: pi.set_servo_pulsewidth(rotation_pin, value) 
                elif code ==  5: pi.set_servo_pulsewidth(elevation_pin, value)
                elif code ==304 and value: servo([top_pin, bottom_pin], [1350, 1950]) 
        except (socket.timeout, struct.error) as e:
            ctrl_ss.shutdown(socket.SHUT_RDWR) 
            ctrl_ss.close()
            print('Failed to receive any data from server so that control channel closes and will reopen in 5 seconds', e)
            time.sleep(5)      
        except ConnectionRefusedError:
            print('Failed to connect the server and retry in 5 seconds')
            time.sleep(5)
#threading.Thread(target=ctrd).start()

In [ ]:
def chksum_nmea(sentence): 
    csum = 0  
    for c in sentence: 
        csum ^= ord(c)
    return str(hex(csum)).upper()[2:] 

def newGPRMC():  
    lat, lon = 4043.831, 7359.70
    for i in range(36,1000):
        lat = lat + 0.003*random.randint(-50,20) 
        lon = lon + 0.003*random.randint(-20,100) 
        timestamp = time.strftime("%H%M%S", time.gmtime())
        chksum = chksum_nmea('GPRMC,{},A,{},N,0{},W,022.4,084.4,210719,003.1,W'.format(timestamp,lat,lon))  
        yield '$GPRMC,{},A,{},N,0{},W,022.4,084.4,210719,003.1,W*{}'.format(timestamp,lat,lon,chksum)
GPRMC = newGPRMC() # gps_simulator, when I need to test tracking system

In [ ]:
class SENSOR(): 
    def __init__(self):
        self.data = {}
        self.Temperature = 0 
        self.interval = 3
        self.sg = serial.Serial(port='/dev/ttyS0', baudrate=9600, timeout=3) # serial commu with gps chip
        sm = serial.Serial(port='/dev/ttyACM0', baudrate=115200, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, timeout=0) # serial commu with microbit
        self.repl = repl.REPL(sm) 
        self.repl.to_raw()
        threading.Thread(target=self.reporter).start()
        time.sleep(3) # wait for the readiness of sensors 
    
    def reporter(self): 
        while True:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sensor_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            sensor_ss.connect((server_ip, sensor_port))
            while True:
                data = {}
                while True:
                    data['P'] = GPRMC.__next__()
                    break
                    if gprmc[0:6] == b'$GPRMC': data['P'] = self.sg.readline() # You can get GPS data only if the on-board LED blinks.  
                self.repl.send_command("print(temperature())") 
                self.Temperature = self.repl.wait_response().rstrip()
                self.repl.send_command("print(accelerometer.get_values())") 
                data['a'] = self.repl.wait_response()
                self.repl.send_command("print(compass.heading())") 
                data['N'] = self.repl.wait_response()
                j = json.dumps(data)
                try:
                    sensor_ss.send(struct.pack('<H', len(j)))
                    sensor_ss.send(j.encode()) 
                except Exception as e:
                    sensor_ss.shutdown(socket.SHUT_RDWR) 
                    sensor_ss.close()
                    print('sensor channel closes and will reopen in 15 seconds', e)
                    time.sleep(15)
                    break   
                time.sleep(self.interval)    

sensor = SENSOR() 

In [ ]:
camera = picamera.PiCamera(resolution=(640, 480)) 
camera.rotation = 270
camera.start_preview() 
time.sleep(2)

# https://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream
# https://picamera.readthedocs.io/en/release-1.13/recipes2.html#web-streaming

def imgd():
    while True:
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            img_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            img_ss.connect((server_ip, img_port)) 
            img_io = img_ss.makefile('wb')
            stream = io.BytesIO()
            for foo in camera.capture_continuous(stream, 'jpeg'): 
                dt = datetime.datetime.now().strftime('@ %H:%M:%S %b %d %Y')
                camera.annotate_text = f'Celsius:{sensor.Temperature.rstrip()} {dt}'  
                img_io.write(struct.pack('<L', stream.tell()))
                img_io.flush()
                stream.seek(0)
                img_io.write(stream.read()) 
                img_io.flush()
                stream.seek(0)
                stream.truncate()
                time.sleep(1)
        except Exception as e:
            print(e)
            time.sleep(15)
    
threading.Thread(target=imgd).start() 